<a href="https://colab.research.google.com/github/shashkovnikita/Telegram_bot/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import random
import nltk
import json
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [74]:
f = open("/content/drive/MyDrive/BOT_CONFIG.json", 'r')
BOT_CONFIG = json.load(f)

In [75]:
texts = []
intent_names = []

for intent, intent_data in BOT_CONFIG['intents'].items():
    for example in intent_data['examples']:
      texts.append(example)
      intent_names.append(intent)

In [45]:
vectorizer = TfidfVectorizer(ngram_range=(2,4), analyzer = 'char')
X = vectorizer.fit_transform(texts)
clf=LinearSVC()
clf.fit(X, intent_names)

LinearSVC()

In [46]:
n = 10
scores=[]
vectorizer = CountVectorizer(ngram_range=(2, 4), analyzer='char')
X = vectorizer.fit_transform(texts)

for i in range (n):
  X_train, X_test, y_train, y_test = train_test_split(X, intent_names)
  clf=LinearSVC()
  clf.fit(X_train, y_train)

  score = clf.score(X_test, y_test)
  print(score)
  scores.append(score)
print(scores)
print('average=',sum(scores) / len(scores))

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


0.5062240663900415


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


0.4979253112033195
0.4785615491009682


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


0.48824343015214383


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


0.5020746887966805
0.4979253112033195
[0.5062240663900415, 0.4979253112033195, 0.4785615491009682, 0.49930843706777317, 0.5034578146611342, 0.4854771784232365, 0.5131396957123098, 0.48824343015214383, 0.5020746887966805, 0.4979253112033195]
average= 0.4972337482710927


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [47]:
# Проверка чистоты датасета значение должно приблизиться к 100% Модель работает и тренируется сама на себе
clf=LinearSVC()
clf.fit(X, intent_names)
clf.score(X, intent_names)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


0.9626168224299065

In [48]:
def clear_text(text):
  text=text.lower()
  text=''.join(char for char in text if char in 'абвгдеёжзийклмнопрстуфхцчшщьыъэюя -abcdefghijklmnopqrstuvwxyz')
  return(text)

In [49]:
def classify_intent(replica):
  replica=clear_text(replica)

  for intent, intent_data in BOT_CONFIG['intents'].items():
    for example in intent_data['examples']:
      example = clear_text(example)
      distance = nltk.edit_distance (replica, example)
      ##print(example,distance, round(distance / len(example),2))

      if len(example) and distance / len(example) < 0.3:
        return(intent)

In [50]:
def classify_intent(replica):
  intent = clf.predict(vectorizer.transform([replica]))[0]
  examples = BOT_CONFIG['intents'][intent]['examples']

  for example in examples:
    example = clear_text(example)
    if len(example)>0:

      if abs(len(example) - len(replica)) / len(example) < 0.5:
        distance = nltk.edit_distance (replica, example)
        if len(example) and distance / len(example) < 0.5:
            return(intent)

In [51]:
def get_answer_by_intent(intent):
  if intent in BOT_CONFIG['intents']:
    responses = BOT_CONFIG['intents'][intent]['responses']
    return random.choice(responses)

### **Generative model**

In [68]:
with open('/content/drive/MyDrive/dialogues.txt') as dialogues_file:
  dialogues_text=dialogues_file.read()
  dialogues = dialogues_text.split('\n\n')

In [53]:
def clear_text(text):
  text=text.lower()
  text=''.join(char for char in text if char in 'абвгдеёжзийклмнопрстуфхцчшщьыъэюя -abcdefghijklmnopqrstuvwxyz')
  return(text)

dataset = []
questions = set()

for dialogue in dialogues:
  replicas = dialogue.split('\n')
  replicas = replicas[:2]

  if len(replicas) ==2:
    question, answer = replicas
    question = clear_text(question [2:])
    answer = answer[2:]

    if len(question)>0 and question not in questions:
      questions.add(question)
      dataset.append([question,answer])

In [54]:
dataset_by_word = {} # {word: [question with word, answer], ...], ...}
for question, answer in dataset:
  words=question.split(' ')
  for word in words:
    if word not in dataset_by_word:
      dataset_by_word[word]=[]
    dataset_by_word[word].append([question,answer])

dataset_by_word_filtered = {}
for word, word_dataset in dataset_by_word.items():
  if len(word_dataset) < 5000:
    dataset_by_word_filtered[word] = word_dataset

In [55]:
dataset_by_word_count=[]
for word, word_dataset in dataset_by_word.items():
  dataset_by_word_count.append([word, len(word_dataset)])
dataset_by_word_count.sort(key=lambda pair: pair[1], reverse=True)

In [56]:
def generate_answer(replica):
  replica=clear_text(replica)
  if not replica:
    return

  words = set(replica.split(' '))
  words_dataset= []

  for word in words:
    if word in dataset_by_word_filtered:
      word_dataset = dataset_by_word_filtered[word]
      words_dataset += word_dataset

  for question, answer in words_dataset:
    if abs(len(question) - len(replica)) / len(question) < 0.2:
      distance = nltk.edit_distance (replica, question)
      if distance / len(question) < 0.2:
        return answer

### **Заглушка**

In [57]:
def get_stub():
  failure_phrases = BOT_CONFIG['failure_phrases']
  #TODO
  return random.choice(failure_phrases)

In [58]:
stats = {'intents': 0, 'generative': 0, 'stubs': 0}

### **ТЕЛО БОТА**

In [59]:
def bot(replica):
  # NLU
  intent = classify_intent(replica)

  # Получение ответа

  # Правила
  if intent:
    answer = get_answer_by_intent(intent)
    if answer:
      stats['intents']+=1
      return(answer)

  #Генеративная модель
  answer = generate_answer(replica)
  if answer:
    stats['generative']+=1
    return(answer)

  #Заглушка
  answer=get_stub()
  stats['stubs']+=1
  return(answer)

In [60]:
stats

{'intents': 0, 'generative': 0, 'stubs': 0}

In [61]:
! pip install python-telegram-bot

In [62]:
! pip install nest_asyncio

In [63]:
import nest_asyncio
nest_asyncio.apply() # Решение ошибки "This event loop is already running"

In [64]:
TOKEN="1033794649:AAFbCXQ3wJGa5ebPAY2ZcGhvXBYJ0Lb7mEs"

In [65]:
from telegram import Update # Обновление пришедшее к нам с серверов ТГ
from telegram.ext import ApplicationBuilder, MessageHandler, filters

# Создаем и настраиваем бот-приложение
app = ApplicationBuilder().token(TOKEN).build()

async def telegram_reply(upd: Update, ctx):
    name = upd.message.from_user.full_name
    user_text = upd.message.text
    print(f"{name}: {user_text}")
    reply = bot(user_text)
    print(f"BOT: {reply}")
    await upd.message.reply_text(reply)

handler = MessageHandler(filters.TEXT, telegram_reply) # Создаем обработчик текстовых сообщений
app.add_handler(handler) # Добавляем обработчик в приложение

# app.run_polling # Наш код регулярно опрашивает сервер на предмет новых Апдейтов
# app.run_webhook # Запускает веб-сервер, к которому будет подключаться сам ТГ и присылать туда апдейты
app.run_polling()

Никита Шашков: Привет
BOT: Здравствуй
Никита Шашков: Кто ты
BOT: Я тот кого трудно найти, легко потерять и невозможно забыть
Никита Шашков: А я кто
BOT: Вера моя, вера, Богом Единым кованная, людьми желанная!
Никита Шашков: Ого
BOT: Это ваши люди?
Никита Шашков: Да
BOT: Что то не понятно
Никита Шашков: Нет
BOT: Хорошо
Никита Шашков: Да
BOT: Я всего лишь бот, сэр
Никита Шашков: Да
BOT: Попробуйте написать подругому.


RuntimeError: Cannot close a running event loop